In [2]:
import pandas as pd
import numpy as np
from random import random
from CTL2.causal_tree_learn import CausalTree
from DGP.dgp6 import dgp_ate4_int
from DGP.dgp6 import dgp_ate4
from MC.mc2 import MC_MSE_2
from MC.mc2_maxd import MC_MSE_2max
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
import matplotlib.pyplot as plt
import pickle as pkl
#from notify_run import Notify 
#notify = Notify()
import datetime

In [16]:
def MC_ate4(n, var_e, reps, nomin_test):    
    #true ATE
    ATE1 = np.ones(reps)*3
    ATE2 = np.ones(reps)*1
    ATE3 = np.ones(reps)*-2
    ATE4 = np.ones(reps)*-3
    #predicted ate
    ate1 = []
    ate2 = []
    ate3 = []
    ate4 = []
    #TOTAL MSE, Bias, Var
    MSE = []
    bias = []
    var = []
    n2 = int(n/2)
    
    leafs = []
    for i in range(reps):
        x_train, x_test, y_train, y_test, treat_train, treat_test = dgp_ate4_int(n,var_e)     
        #true individual treatment effect
        #tau = np.where(((x1 >= 0) & (x2 >= 0)),2, np.where(((x1 >= 0) & (x2 < 0)),1, np.where(((x1 < 0) & (x2 >= 0)),-1,-2)))
        tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0)),3, np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0)),1, np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0)),-2,-3)))
        #predicted infividual treatment effect
        ctl_predict = MC_MSE_2max(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)
        #predicted ATE
        ate1 = np.append(ate1, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] >= 0))]))
        ate2 = np.append(ate2, np.mean(ctl_predict[np.where((x_test[:,0] >= 0) & (x_test[:,1] < 0))]))
        ate3 = np.append(ate3, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] >= 0))]))
        ate4 = np.append(ate4, np.mean(ctl_predict[np.where((x_test[:,0] < 0) & (x_test[:,1] < 0))]))
        
        #leafs = np.append(leafs, len(np.unique(ctl_predict)))
        #nn = int(len(ctl_predict))
        MSE = np.append(MSE, mse(ctl_predict,tau))
        bias = np.append(bias, np.sum(np.subtract(tau,ctl_predict))/n2)
        var = np.append(var, np.var(np.subtract(tau,ctl_predict)))

    #ATE MSE's
    mean_ate = np.array([np.mean(ate1), np.mean(ate2), np.mean(ate3), np.mean(ate4)])    
    mse_ate = np.array([mse(ATE1, ate1), mse(ATE2, ate2),mse(ATE3, ate3), mse(ATE4, ate4)])
    bias_ate = np.array([np.sum(np.subtract(ATE1,ate1))/reps, np.sum(np.subtract(ATE2,ate2))/reps, np.sum(np.subtract(ATE3,ate3))/reps, np.sum(np.subtract(ATE4,ate4))/reps])
    var_ate = np.array([np.var(ate1), np.var(ate2), np.var(ate3), np.var(ate4)])
    
    return mean_ate, mse_ate, bias_ate, var_ate, np.mean(MSE), np.mean(bias), np.mean(var)

In [30]:
print("start time: ", str(datetime.datetime.now())[11:16])
test_sizes = [0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

mean_ate_ = []
mse_ate_ = []
bias_ate_ = []
var_ate_ = []
MSE_ = []
bias_ = []
var_ = []

var_e = 0.1
n = 1000
reps = 500

for size in test_sizes:
    mean_ate, mse_ate, bias_ate, var_ate, MSE, bias, var = MC_ate4(n, var_e, reps, size)
    mean_ate_ = np.append(mean_ate_,mean_ate)
    mse_ate_ = np.append(mse_ate_,mse_ate)
    bias_ate_ = np.append(bias_ate_, bias_ate)
    var_ate_ = np.append(var_ate_, var_ate)
    MSE_ = np.append(MSE_, MSE)
    bias_ = np.append(bias_, bias)
    var_ = np.append(var_, var)
    
#notify.send('Done running the simulations')
print("end time: ", str(datetime.datetime.now())[11:16])

start time:  12:41
end time:  13:11


In [31]:
results = [['var = 0.1, reps = 500, n = 1000, with int.'],[mean_ate_], [mse_ate_], [bias_ate_], [var_ate_], [MSE_], [bias_], [var_]]
with open('te5_r1_.pkl','wb') as f:
    pkl.dump(results, f)

In [ ]:
df = pd.DataFrame([test_sizes, MSE1, MSE2, BIAS1, BIAS2, VAR1, VAR2, MSE_TOTAL, MSEB, MSEV]).transpose().set_index(0)
df.columns=['MSE1', 'MSE2', 'BIAS1', 'BIAS2', 'VAR1', 'VAR2', 'MSE_TOTAL', 'MSE_T_BIAS', 'MSE_T_VAR']

In [2]:
import pickle as pkl
with open('te5_r1.pkl', 'rb') as f:
    df1 = pkl.load(f)

In [4]:
n = 1000
var_e = 0.01
nomin_test = 0.5

x_train, x_test, y_train, y_test, treat_train, treat_test = dgp_ate4_int(n,var_e)     
#true individual treatment effect
#tau = np.where(((x1 >= 0) & (x2 >= 0)),2, np.where(((x1 >= 0) & (x2 < 0)),1, np.where(((x1 < 0) & (x2 >= 0)),-1,-2)))
tau = np.where(((x_test[:,0] >= 0) & (x_test[:,1] >= 0)),3, np.where(((x_test[:,0] >= 0) & (x_test[:,1] < 0)),1, np.where(((x_test[:,0] < 0) & (x_test[:,1] >= 0)),-2,-3)))
#predicted infividual treatment effect
ctl_predict = MC_MSE_2max(x_train, x_test, y_train, y_test, treat_train, treat_test, nomin_test)


In [8]:
np.sum(np.subtract(tau,ctl_predict))/500

-0.023116228941507548

In [9]:
mse(tau,ctl_predict)

0.6381846687674916

In [10]:
np.var(np.subtract(tau,ctl_predict))

0.6376503087270152

In [11]:
np.square(np.sum(np.subtract(tau,ctl_predict))/500) + np.var(np.subtract(tau,ctl_predict))

0.6381846687674915